### The purpose of this script is to combine the individual model predictions into an ensemble prediction


In [39]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score

# load evaluation data
test_df = pd.read_csv("C:/Data/Kaggle EyePACS/test_public.csv")
labels = np.array(test_df['level'])

### Function to threshold predictions

In [32]:

def threshold_predictions(preds):

    predictions = preds.copy()
            
    # Convert MSE floats to integer predictions
    predictions[predictions < 0.5] = 0
    predictions[(predictions >= 0.5) & (predictions < 1.5)] = 1
    predictions[(predictions >= 1.5) & (predictions < 2.5)] = 2
    predictions[(predictions >= 2.5) & (predictions < 3.5)] = 3
    predictions[(predictions >= 3.5) & (predictions < 10000000)] = 4
    predictions = predictions.astype(int)
   
            
    return predictions

### Compute the ensemble predictions using/not using TTA

In [42]:
for tta in [False, True]:
    pred_values = None
    total_models = 0
    for model_type in ['b3', 'b4', 'b5']:
        for ensemble_iter in ['0', '1']:
            total_models += 1
            
            output_file_path = model_type + '_' + str(ensemble_iter)
            if tta == True:
                data_path = 'predictions/' + output_file_path + '_tta.csv'
            else:
                data_path = 'predictions/' + output_file_path + '.csv'
                
                
            pred_df = pd.read_csv(data_path)
            
            if pred_values is None:
                pred_values = np.array(pred_df['level'])
            else:
                pred_values += np.array(pred_df['level'])
                
    pred_values/=total_models
    preds = threshold_predictions(pred_values)
    
    print(f"QuadraticWeightedKappa TTA: {tta}: {cohen_kappa_score(labels, preds, weights='quadratic')}")

        
    

QuadraticWeightedKappa TTA: False: 0.812463152736897
QuadraticWeightedKappa TTA: True: 0.8141355353993828
